Link Videos: https://posinatel-my.sharepoint.com/:f:/g/personal/felipe_gabriel_ges_inatel_br/EtcPzPbtPGdEs5ioqVgwNJoBIIeFOALExlmHNc_Zw_NJrg?e=XQYZSn

ou

Video Inicial: https://youtu.be/w31XvXe3Rgw
Video Final: https://youtu.be/Uy-V6I--8jQ

## Plano Detalhado

## -> Bibliotecas utilizadas:

moviepy.editor (edição de video/audio) -> pip install moviepy

numpy (operações) -> pip install numpy

## 1. gravar um video de si mesmo lendo o texto no arquivo anexo

In [1]:
#libs necessarias
import moviepy.editor as mpe
import numpy as np

In [2]:
#carregando video
vd = mpe.VideoFileClip("videoInicial.mp4")

## 2. inverter a imagem, no eixo X, a cada 20 segundo

Dividir o vídeo em segmentos de 20s

Aplicar mirrorX (espelhamento no eixo X) em blocos alternados

Exemplo:

0s–20s → Normal

20s–40s → Espelhado

40s–60s → Normal

e assim por diante

Concatenar todos os segmentos em um

In [4]:
def mirrorX(video):
    """
    Divide o video em blocos de 20s
    Aplica espelhamento horizontal (eixo X) alternadamente em cada bloco
    Concatena tudo
    """
    duration = video.duration
    cuts = []
    mirror = False

    for start in np.arange(0, duration, 20):
        end = min(start + 20, duration)
        subclip = video.subclip(start, end)
        
        if mirror:
            subclip = subclip.fx(mpe.vfx.mirror_x)
            
        cuts.append(subclip)
        mirror = not mirror

    return mpe.concatenate_videoclips(cuts)

vd2 = mirrorX(vd)

## 3. diminiur o som, de forma gradativa, a cada 30 segundos, de forma que os últimos 10 segundos fique em silêncio (sem som)

Dividir o áudio em blocos de 30s.

Diminuir progressivamente o volume (100% → 10%) conforme a quantidade de blocos.

Exemplo:

Duraçao = 2:18 = 138s -> 138 - 10 = 128s

Blocos = 128s / 30s = 4,26 = 4 (arredendamento)

Diferença de volume = 100% - 10% = 90% = 0.9

Redução por bloco = 0.9 / (4 - 1) = 0.3 = 30s

0s – 30s           → 1.0 - (0.3 × 0) = 1.0	100%

30s – 60s          → 1.0 - (0.3 × 1) = 0.7	70%

60s – 90s          → 1.0 - (0.3 × 2) = 0.4	40%

90s – 120s         → 1.0 - (0.3 × 3) = 0.1	10%

120s – 128s(resto) → mantem o ultimo (0.1)	10%

ultimos 10s → mudo


In [5]:
def fadeOut(video):
    '''
    Divide o audio em blocos de 30s
    Para cada bloco, reduz gradualmente o volume conforme o numero de blocos
    Muta os 10s finais
    Concatena tudo
    '''
    #verificando se existe audio
    if video.audio is None:
        return video
    
    duration = video.duration
    noMuted = duration - 10  #remover 10s para mutar
    blockDuration = 30
    cuts = [] #lista para armazenar os audios
    currentTime = 0 #contador para acompanhar o tempo atual do video
    
    while currentTime < noMuted:
        endTime = min(currentTime + blockDuration, noMuted)
        blockIndex = int(currentTime // blockDuration) #calcular  o indice do bloco
        totalBlocks = int(np.ceil(noMuted / blockDuration)) #numero total de blocos antes de mutar
        
        #calcular o volume do bloco, diminuindo gradualmente de 100% para 10%
        if totalBlocks <= 1:
            volume = 1.0
        else:
            volume = max(0.1, 1.0 - 0.9 * (blockIndex / (totalBlocks - 1))) 
        
        #extrair e modificar audios
        cut = video.audio.subclip(currentTime, endTime) #trecho de audio do bloco atual
        cut = cut.volumex(volume) #modifica volume do trecho
        cuts.append(cut) #adiciona a lista
        
        currentTime = endTime #atualiza o tempo para o duration do bloco
    
    #mutar 
    if noMuted < duration:
        silent_cut = video.audio.subclip(noMuted, duration) #extrai ultimos 10s
        silent_cut = silent_cut.volumex(0) #volume 0
        cuts.append(silent_cut) #adiciona a lista
    
    #concatenar audios
    fadeOutAudio = mpe.concatenate_audioclips(cuts)
    
    return video.set_audio(fadeOutAudio)

vd3 = fadeOut(vd2)

## 4. após o primeiro minuto, 60 segundos, fazer um corte para o segundo 80

In [ ]:
    duration = vd.duration
    cut1 = vd3.subclip(0, 60)
    cut2 = vd3.subclip(60, 80) #cortar de 60 a 80s
    cut3  = vd3.subclip(80, duration) #cortar de 80 ao final

## 5. por fim, insira os 20 segundos cortados acima no fim do video

In [7]:
#concatenar em sequencia -> 0-60 + 80-final + 60-80
videoFinal = mpe.concatenate_videoclips([cut1, cut3, cut2])

## 6. exibir e salvar vídeo

In [ ]:
videoFinal.ipython_display(width=480, maxduration=380)
videoFinal.write_videofile('videoFinal.mp4')

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
Moviepy - Building video videoFinal.mp4.
MoviePy - Writing audio in videoFinalTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video videoFinal.mp4



Moviepy - Done !
Moviepy - video ready videoFinal.mp4
